In [2]:
# Imports
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import h3
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVR, LinearSVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.optimizers import Adam
import papermill as pm
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
import concurrent.futures
import random
import os 
from tensorflow import keras
from tensorflow.keras import layers
import copy
import tensorflow as tf

In [3]:
# Constants
RANDOM_STATE=42

In [4]:
def preprocess_data(df, train_ratio=0.75, validation_ratio=0.15):
    features = ['demand_h-2', 'demand_h-24', 'hour_sin', 'hour_cos', 'weekend', 'season_sin', 'season_cos', 'public_holiday', 'temperature', 'precip']
    features_to_scale = ['demand_h-2', 'demand_h-24', 'temperature', 'precip']
    target = 'demand'
    
    # Copy the input DataFrame
    df_copy = df.copy()

    # Select features and target
    X = df_copy[features]
    y = df_copy[target]

    # Split into train, validation, and test sets
    test_ratio = (1-train_ratio)-validation_ratio
    X_train_unscaled, X_test_unscaled, y_train, y_test = train_test_split(X, y, test_size=(1 - train_ratio), random_state=RANDOM_STATE)
    X_val_unscaled, X_test_unscaled, y_val, y_test = train_test_split(X_test_unscaled, y_test, test_size=test_ratio / (validation_ratio + test_ratio), random_state=RANDOM_STATE)

    # Scaling
    scaler = StandardScaler()
    scaler.fit(X_train_unscaled[features_to_scale])

    X_train = X_train_unscaled.copy()
    X_val = X_val_unscaled.copy()
    X_test = X_test_unscaled.copy()

    X_train[features_to_scale] = scaler.transform(X_train_unscaled[features_to_scale])
    X_val[features_to_scale] = scaler.transform(X_val_unscaled[features_to_scale])
    X_test[features_to_scale] = scaler.transform(X_test_unscaled[features_to_scale])

    return (X_train, X_val, X_test, y_train, y_val, y_test)

In [5]:
# Pull Datasets from Feature Engineering
# This takes around ~ 25-35 Minutes and will fill you RAM and CPU nearly completely. 

# Constants
TIME_RESOLUTIONS = ['1H', '2H', '6H', '24H']
SPATIAL_RESOLUTIONS = [6, 7, 8]
DATASET_SUFFIX = ['_h3', '_census']
PROCESSING_NOTEBOOK_FILE = './predicitve_feature_engineering.ipynb'
FILE_BASE_NAME='./data/predictive/dataset'
RUN_FEATURE_ENGINERRING = False

In [6]:
%%capture
# ^ supress notebook outputs as to not get spammed by 12 Data preparation notebooks. Output can be found under /data/notebook_outs

output_filenames = []

# Function to execute a notebook with given parameters
def execute_notebook(notebook, params):
    output_notebook = f"./data/notebook_outs/output_{random.randint(1, 100)}"
    pm.execute_notebook(notebook, output_notebook, parameters=params)

# Generate notebooks and parameters
notebooks_and_params = []
for time_res in TIME_RESOLUTIONS:
    for spatial_res in SPATIAL_RESOLUTIONS:
        output_filename_base = f'{FILE_BASE_NAME}-spatial_{spatial_res}-temporal_{time_res}'
        output_filenames.append(output_filename_base)

        notebook = PROCESSING_NOTEBOOK_FILE  # Replace with your notebook filename
        params = {
            "TIME_RESOLUTION": time_res,
            "SPATIAL_RESOLUTION": spatial_res,
            "OUTPUT_FILENAME_BASE": output_filename_base
        }
        notebooks_and_params.append((notebook, params))

if (RUN_FEATURE_ENGINERRING):
    
    # Max 4 on 32 GB Ram (Adrians Machine)
    MAX_WORKER_THREADS = 4
    # Parallel execution using concurrent.futures
    with concurrent.futures.ThreadPoolExecutor(MAX_WORKER_THREADS) as executor:
        futures = [executor.submit(execute_notebook, nb, params) for nb, params in notebooks_and_params]

    # Wait for all futures to complete
    concurrent.futures.wait(futures)

    # Print exception details
    for future in futures:
        exception = future.exception()
        if exception:
            print(f"Exception in future: {exception}")


In [7]:

# Import datasets
datasets = {}

for filepath in output_filenames:
    filename = f'{os.path.basename(filepath)}_census'
    datasets[filename] = preprocess_data(pd.read_csv(f'{filepath}_census.csv'))

    filename = f'{os.path.basename(filepath)}_h3'
    datasets[filename] = preprocess_data(pd.read_csv(f'{filepath}_h3.csv'))

### Function for Hyperparameter Tuning
This function enables us to do hyperparameter tuning for any model in the sklearn universe. We have the choice to either do a RandomizedGridSearch with cross validation or a standard GridSearch, the latter is computationally heavier.

In [24]:
def optimize_hyperparameters(param_grid, model, X, y, randomized=False):
    if randomized:
        grid = RandomizedSearchCV(model, param_grid)
    else:
        grid = GridSearchCV(model, param_grid, verbose=3)

    grid.fit(X, y)
    print(f"Best params: {grid.best_params_}")
    print(f"Scoring: {grid.best_score_}")
    return grid


### Model Evaluation Function

In [25]:
# Model Evaluation function:
def evaluate_model(y, y_pred):
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    return rmse, mae, r2

## SVR Prediction

We will now do SVR to estimate demand given our features. The issue is that the implementation of our SVR is based on libsvm. The fit time complexity is more than quadratic with the number of samples which makes it hard to scale to datasets with more than a couple of 10000 samples. For large datasets, we can either downsample or use sklearn.svm.LinearSVR which has a more performant implementation. Libsvm scales either O(n_features * n_samples^2) or O(n_features * n_samples^3). We will therefore focus our eperimentations on the linear kernel and only briefly try the other ones with reduced dataset sizes.

Overview:


In [26]:
# Hyperparameters - General
C=1
EPSILON=0.1

# Hyperparameters - Linear
MAX_ITER = 2500 # default 1000

POLY_DEGREE=3

# Training Parameters
CACHE_SIZE = 2048 # in MB

# Parallel Training 
# rougly 4 Min per Model, can run ~ 8 in parallel -> 30 Min for LinSVR Training
MAX_WORKER_THREADS = 8

In [ ]:
def train_linear_svr_models(dataset_name, dataset, C, EPSILON):
    X_train, X_val, X_test, y_train, y_val, y_test = dataset

    # Dual = automatically select the fastest problem (dual or primary problem) based on #features vs #values
    svr_lin = LinearSVR(C=C, epsilon=EPSILON, dual='auto', max_iter=MAX_ITER)
    
    print(f'Fitting for {dataset_name}..')
    svr_lin.fit(X_train, y_train)

    print(f'Predicting for {dataset_name}..')
    y_lin = svr_lin.predict(X_test)

    print(f'Evaluating for {dataset_name}..')
    svr_metrics[dataset_name] = evaluate_model(y_test, y_lin)
    svr_models[dataset_name] = svr_lin


def train_with_dataset_and_parameters(dataset_key, dataset_values):
    train_linear_svr_models(dataset_key, dataset_values, C, EPSILON)

svr_metrics = {}
svr_models = {}

# Parallel execution using concurrent.futures
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKER_THREADS) as executor:
    futures = [executor.submit(train_with_dataset_and_parameters, dataset_name, dataset_values)
               for dataset_name, dataset_values in datasets.items()]

# Wait for all futures to complete
concurrent.futures.wait(futures)

for future in futures:
    exception = future.exception()
    if exception:
        print(f"Exception in future: {exception}")

## Non-Linear SVR
As mentioned previously, we will just briefly venture into the realm of alternative kernels, as the library does not offer an efficient implmentation for rbf, sigmoid or poly kernels. With a non-linear runtime, we have to massively cut down on our dataset to get a reasonable training time. The documentation of the scikit-learn implementation of the SVR already suggests the use-cases of more complex kernel functions: If you have a limited dataset "at most a couple of 10000" (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR) and want to maximize the performance, using an more complex kernel function like rbf can be beneficial. However, if a large amount of data is available, in the case of the dataset present, it is more effective to train with the full dataset on the linear kernel, instead of using a more intricate kernel with a smaller dataset.

To generate a reduced trainingset, we will run the train_test_split function again and discard leftover data. We will run half of the datasets with the poly and the other half with the rbf kernel

In [11]:
# Hyperparameters - General
C=1
EPSILON=0.1

# Hyperparameters - Linear
MAX_ITER = 2500 # default 1000

POLY_DEGREE=3

# Training Parameters
CACHE_SIZE = 2048 # in MB
MAX_X_TRAIN_REDUCED_SIZE = 50000
# Parallel Training 
# rougly 4 Min per Model, can run ~ 8 in parallel -> 30 Min for LinSVR Training
MAX_WORKER_THREADS = 8

# split datasets
def random_split_dict(dictionary, split_ratio=0.5):
    copied_dict = copy.deepcopy(dictionary)
    
    keys = list(copied_dict.keys())
    random.shuffle(keys)
    
    split_index = int(len(keys) * split_ratio)
    
    dict_part1 = {key: copied_dict[key] for key in keys[:split_index]}
    dict_part2 = {key: copied_dict[key] for key in keys[split_index:]}
    
    return dict_part1, dict_part2

datasets_reduced_rbf, datasets_reduced_poly = random_split_dict(datasets)

In [ ]:
# RBF
def train_rbf_svr_models(dataset_name, dataset, C, EPSILON):
    X_train, X_val, X_test, y_train, y_val, y_test = dataset

    # Calculate the test_size dynamically based on X_train size
    if len(X_train) <= MAX_X_TRAIN_REDUCED_SIZE:
        test_size = 0.0  # No test set if X_train is smaller than or equal to MAX_X_TRAIN_REDUCED_SIZE
    else:
        test_size = 1.0 - (MAX_X_TRAIN_REDUCED_SIZE / len(X_train))

    # reduce dataset size
    X_train_reduced, X_test_reduced, y_train_reduced, y_test_reduced = train_test_split(X_train, y_train, test_size=test_size, random_state=RANDOM_STATE)

    # Dual = automatically select the fastest problem (dual or primary problem) based on #features vs #values
    svr_rbf = SVR(kernel='rbf', C=C, epsilon=EPSILON, max_iter=MAX_ITER, cache_size=2048)
    
    print(f'Fitting for {dataset_name}..')
    svr_rbf.fit(X_train_reduced, y_train_reduced)

    print(f'Predicting for {dataset_name}..')
    y_preds = svr_rbf.predict(X_test_reduced)

    print(f'Evaluating for {dataset_name}..')
    svr_metrics[dataset_name] = evaluate_model(y_test_reduced, y_preds)
    svr_models[dataset_name] = svr_rbf


def train_with_dataset_and_parameters(dataset_key, dataset_values):
    train_rbf_svr_models(dataset_key, dataset_values, C, EPSILON)

rbf_metrics = {}
rbf_models = {}

# Parallel execution using concurrent.futures
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKER_THREADS) as executor:
    futures = [executor.submit(train_with_dataset_and_parameters, dataset_name, dataset_values)
               for dataset_name, dataset_values in datasets_reduced_rbf.items()]

# Wait for all futures to complete
concurrent.futures.wait(futures)

for future in futures:
    exception = future.exception()
    if exception:
        print(f"Exception in future: {exception}")

In [ ]:
#  Poly
def train_poly_svr_models(dataset_name, dataset, C, EPSILON):
    X_train, X_val, X_test, y_train, y_val, y_test = dataset

     # Calculate the test_size dynamically based on X_train size
    if len(X_train) <= MAX_X_TRAIN_REDUCED_SIZE:
        test_size = 0.001  # No test set if X_train is smaller than or equal to MAX_X_TRAIN_REDUCED_SIZE
    else:
        test_size = 1.0 - (MAX_X_TRAIN_REDUCED_SIZE / len(X_train))

    # reduce dataset size
    X_train_reduced, X_test_reduced, y_train_reduced, y_test_reduced = train_test_split(X_train, y_train, test_size=test_size, random_state=RANDOM_STATE)

    # Dual = automatically select the fastest problem (dual or primary problem) based on #features vs #values
    svr_poly = SVR(kernel='poly', C=C, epsilon=EPSILON, max_iter=MAX_ITER, cache_size=2048)
 
    print(f'Fitting for {dataset_name}..')
    svr_poly.fit(X_train_reduced, y_train_reduced)

    print(f'Predicting for {dataset_name}..')
    y_preds = svr_poly.predict(X_test_reduced)

    print(f'Evaluating for {dataset_name}..')
    svr_metrics[dataset_name] = evaluate_model(y_test_reduced, y_preds)
    svr_models[dataset_name] = svr_poly


def train_with_dataset_and_parameters(dataset_key, dataset_values):
    train_poly_svr_models(dataset_key, dataset_values, C, EPSILON)

poly_metrics = {}
poly_models = {}

# Parallel execution using concurrent.futures
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKER_THREADS) as executor:
    futures = [executor.submit(train_with_dataset_and_parameters, dataset_name, dataset_values)
               for dataset_name, dataset_values in datasets_reduced_poly.items()]

# Wait for all futures to complete
concurrent.futures.wait(futures)

for future in futures:
    exception = future.exception()
    if exception:
        print(f"Exception in future: {exception}")

## NN Prediction

In [36]:
# Hyperparameters
EPOCHS=10
BATCH_SIZE=128
LEARNING_RATE=0.0025 # higher learn rate as we have a bad gpu
OPTIMIZER=keras.optimizers.Adam(learning_rate=LEARNING_RATE)
LOSS='mean_squared_error'


# Training Multiprocessing
# It seams on a 16 core cpu we can increase this to more than 8
MAX_WORKER_THREADS = 12

In [ ]:
# Model Architecture

# as they are all the same and have no order in the dict we will just grab any element and get the shape of the train dataframe
X_train, X_val, X_test, y_train, y_val, y_test = datasets[random.choice(list(datasets.keys()))]

model_abstract = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),  # Input layer for time series data
    layers.Dense(128, activation='relu'),     # Hidden layer 1
    layers.Dropout(0.3),                      # Dropout layer for regularization
    layers.Dense(64, activation='relu'),      # Hidden layer 2
    layers.Dropout(0.2),                      # Dropout layer for regularization
    layers.Dense(32, activation='relu'),      # Hidden layer 3
    layers.Dropout(0.1),                      # Dropout layer for regularization
    layers.Dense(1)                           # Output layer
])
model_abstract.compile(optimizer=OPTIMIZER, loss=LOSS)

# Training Loop

def train_with_dataset_and_parameters(dataset_name, dataset_values):
    X_train, X_val, X_test, y_train, y_val, y_test = dataset_values
    
    model = keras.models.clone_model(model_abstract)

    OPTIMIZER.build(model.trainable_variables)
    model.compile(optimizer=OPTIMIZER, loss=LOSS)

    model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

    y_preds = model.predict(X_test)

    nn_metrics[dataset_name] = evaluate_model(y_test, y_preds)
    nn_models[dataset_name] = model
    nn_importance[dataset_name] = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)


nn_metrics = {}
nn_models = {}
nn_importance = {}

# Parallel execution using concurrent.futures
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKER_THREADS) as executor:
    futures = [executor.submit(train_with_dataset_and_parameters, dataset_name, dataset_values)
               for dataset_name, dataset_values in datasets_reduced_poly.items()]

# Wait for all futures to complete
concurrent.futures.wait(futures)

for future in futures:
    exception = future.exception()
    if exception:
        print(f"Exception in future: {exception}")
